In [1]:
import pickle
import torch
import torch.nn.functional as F
from torch.nn import Linear

import torch_geometric as pyg
from torch_geometric.loader import DataLoader
from torch_geometric.nn import TopKPooling, GraphConv, global_mean_pool

In [2]:
with open('data/train_pyg.pkl','rb') as ftrain:
  train_dataset=pickle.load(ftrain)
with open('data/test_pyg.pkl','rb') as ftest:
  test_dataset=pickle.load(ftest)


In [3]:
batch_size=32
test_loader = DataLoader(test_dataset, batch_size=batch_size)
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)

In [4]:
class Net(torch.nn.Module):
    def __init__(self, num_features, num_classes, num_layers, hidden, ratio=0.8, dropout=0):
        super().__init__()
        self.conv1 = GraphConv(num_features, hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(hidden, hidden, aggr='mean')
            for i in range(num_layers - 1)
        ])
        self.lin1 = Linear(num_layers * hidden, hidden)
        self.lin2 = Linear(hidden, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x.to_dense(), data.edge_index, data.batch
        edge_weight = None
        x = F.relu(self.conv1(x, edge_index))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = conv(x=x, edge_index=edge_index, edge_weight=edge_weight)
            x = F.relu(x)
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, edge_weight, batch, _ = pool(
                    x=x, edge_index=edge_index, edge_weight=edge_weight,
                    batch=batch)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=Net(1443,2,4,64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-4)

def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y.view(-1))
        loss.backward()
        loss_all += data.y.size(0) * float(loss)
        optimizer.step()
    return loss_all / len(train_dataset)
@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0
    loss_all=0

    for data in loader:
        data = data.to(device)
        pred = model(data)
        loss = F.nll_loss(pred, data.y.view(-1))
        loss_all += data.y.size(0) * float(loss)
        correct += int(pred.max(dim=1)[1].eq(data.y.view(-1)).sum())

    return loss_all / len(loader.dataset), correct / len(loader.dataset)

In [9]:
best_test_acc = test_acc = 0
best_test_loss = float('inf')
patience = start_patience = 30
for epoch in range(1, 1000):
    train_loss = train(epoch)
    _, train_acc = test(train_loader)
    test_loss, test_acc = test(test_loader)
    if test_loss < best_test_loss:        
        best_test_loss=test_loss
        best_test_acc = test_acc
        patience = start_patience
        best_epoch =  epoch
    else:
        patience -= 1
        if patience == 0:
            break
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.3f}, '
          f'Train Acc: {train_acc:.3f}, '
          f'Test Loss: {test_loss:.3f}, '
          f'Test Acc: {test_acc:.3f}')


Epoch: 001, Train Loss: 0.032, Train Acc: 0.990, Test Loss: 0.408, Test Acc: 0.961
Epoch: 002, Train Loss: 0.041, Train Acc: 0.988, Test Loss: 0.385, Test Acc: 0.965
Epoch: 003, Train Loss: 0.035, Train Acc: 0.985, Test Loss: 0.420, Test Acc: 0.960
Epoch: 004, Train Loss: 0.045, Train Acc: 0.991, Test Loss: 0.382, Test Acc: 0.966
Epoch: 005, Train Loss: 0.029, Train Acc: 0.991, Test Loss: 0.395, Test Acc: 0.968
Epoch: 006, Train Loss: 0.028, Train Acc: 0.991, Test Loss: 0.418, Test Acc: 0.968
Epoch: 007, Train Loss: 0.037, Train Acc: 0.989, Test Loss: 0.428, Test Acc: 0.965
Epoch: 008, Train Loss: 0.031, Train Acc: 0.991, Test Loss: 0.446, Test Acc: 0.968
Epoch: 009, Train Loss: 0.028, Train Acc: 0.991, Test Loss: 0.436, Test Acc: 0.966
Epoch: 010, Train Loss: 0.029, Train Acc: 0.991, Test Loss: 0.445, Test Acc: 0.965
Epoch: 011, Train Loss: 0.027, Train Acc: 0.992, Test Loss: 0.463, Test Acc: 0.963
Epoch: 012, Train Loss: 0.036, Train Acc: 0.991, Test Loss: 0.443, Test Acc: 0.965
Epoc

In [10]:
print(f'The Best Epoch: {best_epoch:03d},Test Loss: {best_test_loss:.3f}, Test Acc: {best_test_acc:.3f}')

The Best Epoch: 014,Test Loss: 0.326, Test Acc: 0.966
